In [1]:
import pandas as pd
from transformers import pipeline

# Load the BART large MNLI model for zero-shot classification
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define possible emotion labels
emotion_labels = [
    "anger", "anticipation", "joy", "trust",
    "fear", "surprise", "sadness", "disgust"
]


# Example dataframe
data = {'text': [
    "I am so excited about the future!",
    "This is such a disappointing experience.",
    "I'm terrified of what might happen next.",
]}
df = pd.DataFrame(data)

# Function to classify emotions
def classify_emotion(text, labels):
    result = classifier(text, candidate_labels=labels)
    return result['labels'][0]  # Return the top label

# Apply the classification to each row in the dataframe
df['emotion'] = df['text'].apply(lambda x: classify_emotion(x, emotion_labels))

# Print the resulting dataframe
print(df)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-21 10:10:13.981824: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining mo

                                       text       emotion
0         I am so excited about the future!  anticipation
1  This is such a disappointing experience.       disgust
2  I'm terrified of what might happen next.          fear
